In [6]:
file_path = '/Users/lfinkbeiner/Documents/GitHub/HERA/'
beam_origin = 'HERA_4.9m_E-pattern_151MHz.txt'
beam_destination = 'ant1_s1'

In [7]:
%matplotlib notebook
from RIMEz import beam_models
import numpy as np
from spin1_beam_model import cst_processing, jones_matrix_field

In [8]:
# "The fields are not normalized here, they are to be normalized once a global
# model is fit to these data." what global model?

processor = cst_processing.CSTDataProcessor(
    [file_path + beam_origin,
    file_path + beam_origin,
    file_path + beam_origin],
    np.array([150e6, 151e6, 152e6]),
    1, 1e-4
    # I do not know what the spatial sampling and data uncertainty are here
)


In [9]:
processor.compute_spin1_harmonics()

In [10]:
# We need to write this to a file, because we unfortunately cannot
# pass the data directly to the next class
processor.write_model_data(file_path, beam_destination)

In [11]:
spline_beam_func = beam_models.model_data_to_spline_beam_func(file_path + beam_destination + '.h5', np.array([150e6, 151e6, 152e6]))

//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [12]:
#spline_beam_func(nu, alt, az)
test_out = spline_beam_func(151e6,
                np.array([-np.pi / 4, 0, np.pi / 4]),
                np.array([0, np.pi / 2, np.pi])
)

In [13]:
# to-do: I do not know if the spline takes angles in degrees or radians,
    # I have assumed radians
print(test_out)

array([[[ 0.00000000e+00+0.00000000e+00j,
         -0.00000000e+00+0.00000000e+00j],
        [ 0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j]],

       [[ 0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j],
        [-0.00000000e+00+0.00000000e+00j,
          0.00000000e+00+0.00000000e+00j]],

       [[-2.76584962e-05-2.06096872e-06j,
          2.28540239e-02-3.19287792e-02j],
        [ 3.22238330e-02-7.87022980e-03j,
         -1.53217075e-06-7.50443615e-07j]]])

In [55]:
#spline_beam_func(nu, alt, az)
J = spline_beam_func(151e6,
                np.array([np.pi / 4]),
                np.array([np.pi])
)[0]
J_outer = np.kron(J, np.conj(J))

In [56]:
print('J matrix:')
print(J)
print()

print('Kronecker Product:')
print(J_outer)
print(J_outer.shape)

J matrix:
[[-2.76584962e-05-2.06096872e-06j  2.28540239e-02-3.19287792e-02j]
 [ 3.22238330e-02-7.87022980e-03j -1.53217075e-06-7.50443615e-07j]]

Kronecker Product:
[[ 7.69240002e-10+0.00000000e+00j -5.66303717e-07-9.30203445e-07j
  -5.66303717e-07+9.30203445e-07j  1.54175335e-03+0.00000000e+00j]
 [-8.75042463e-07-2.84091032e-07j  4.39241795e-11-1.75983859e-11j
   9.87731078e-04-8.49001227e-04j -1.10555184e-08+6.60709978e-08j]
 [-8.75042463e-07+2.84091032e-07j  9.87731078e-04+8.49001227e-04j
   4.39241795e-11+1.75983859e-11j -1.10555184e-08-6.60709978e-08j]
 [ 1.10031593e-03+0.00000000e+00j -4.34662505e-08+3.62407056e-08j
  -4.34662505e-08-3.62407056e-08j  2.91071282e-12+0.00000000e+00j]]
(4, 4)


In [57]:
S = .5 * np.array([
        [1, 1, 0, 0],
        [0, 0, 1, 1j],
        [0, 0, 1, -1j],
        [1, -1, 0, 0]
    ])

In [58]:
print('S matrix:')
print(S)
print()

print('S inverse:')
print(np.linalg.inv(S))

S matrix:
[[ 0.5+0.j   0.5+0.j   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0.5+0.j   0. +0.5j]
 [ 0. +0.j   0. +0.j   0.5+0.j   0. -0.5j]
 [ 0.5+0.j  -0.5+0.j   0. +0.j   0. +0.j ]]

S inverse:
[[ 1.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 1.+0.j  0.-0.j  0.-0.j -1.-0.j]
 [ 0.+0.j  1.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.-1.j  0.+1.j  0.+0.j]]


In [59]:
A = np.dot(S, np.dot(J_outer, np.linalg.inv(S)))

In [65]:
print('A matrix:')
print(A)
print(A.shape)

A matrix:
[[-7.20266508e-07-6.07156038e-07j  4.93615423e-04-1.19490666e-03j
   4.93549352e-04+3.46835635e-04j -1.54006715e-07+3.23065006e-07j]
 [ 4.93409897e-04+9.74778890e-04j -1.48917287e-08-1.61965669e-08j
   5.11763584e-08-2.72520853e-08j -4.94284940e-04+1.25821129e-04j]
 [ 4.93446138e-04-1.25493571e-04j -5.11353449e-08+2.72696836e-08j
   1.49385636e-08+1.62141652e-08j -4.94321181e-04-9.74538265e-04j]
 [ 1.54732031e-07-3.23047407e-07j -4.94181726e-04-3.45916487e-04j
  -4.94115655e-04+1.19584792e-03j  7.21079672e-07+6.07138440e-07j]]
(4, 4)


In [63]:
def A_matrix(freq, alt, az):
    J = spline_beam_func(freq, alt, az)[0]
    J_outer = np.kron(J, np.conj(J))
    return np.dot(S, np.dot(J_outer, np.linalg.inv(S)))

In [66]:
test = A_matrix(151e6, np.array([np.pi / 4]), np.array([np.pi])) 
print(test)

[[-7.20266508e-07-6.07156038e-07j  4.93615423e-04-1.19490666e-03j
   4.93549352e-04+3.46835635e-04j -1.54006715e-07+3.23065006e-07j]
 [ 4.93409897e-04+9.74778890e-04j -1.48917287e-08-1.61965669e-08j
   5.11763584e-08-2.72520853e-08j -4.94284940e-04+1.25821129e-04j]
 [ 4.93446138e-04-1.25493571e-04j -5.11353449e-08+2.72696836e-08j
   1.49385636e-08+1.62141652e-08j -4.94321181e-04-9.74538265e-04j]
 [ 1.54732031e-07-3.23047407e-07j -4.94181726e-04-3.45916487e-04j
  -4.94115655e-04+1.19584792e-03j  7.21079672e-07+6.07138440e-07j]]


In [71]:
# absolute values. Here the problem is more obvious:
# from the paper: "...the off-diagonal elements of the A matrix are much smaller than the diagonal elements..."
print(np.absolute(test))

[[9.42030943e-07 1.29284883e-03 6.03229575e-04 3.57895328e-07]
 [1.09254163e-03 2.20020990e-08 5.79801329e-08 5.10047604e-04]
 [5.09153933e-04 5.79522143e-08 2.20467647e-08 1.09273888e-03]
 [3.58192167e-07 6.03219525e-04 1.29390978e-03 9.42641490e-07]]
